In [1]:
import os
import boto3
import pandas as pd

In [2]:
DB_ACCESS_KEY = pd.read_csv('DB_ACCESS_KEY.csv')
aws_access_key_id = DB_ACCESS_KEY['Access key ID'][0]
aws_secret_access_key = DB_ACCESS_KEY['Secret access key'][0]

In [3]:
dynamodb = boto3.resource('dynamodb', region_name='us-east-2', aws_access_key_id=aws_access_key_id, aws_secret_access_key=aws_secret_access_key)
table = dynamodb.Table('StockSentiment')
# keep scanning until we have all the data in the table
response = table.scan()
data = response['Items']
while 'LastEvaluatedKey' in response:
    response = table.scan(ExclusiveStartKey=response['LastEvaluatedKey'])
    data.extend(response['Items'])
# convert the data to a pandas dataframe
sentiment_ticker_list = pd.DataFrame(data)
# convert Date column to datetime
sentiment_ticker_list['Date'] = pd.to_datetime(sentiment_ticker_list['Date'])
# make the index the Date column
sentiment_ticker_list = sentiment_ticker_list.set_index('Date').sort_index(ascending=False)
sentiment_ticker_list

,ticker_sentiment_score,ticker_sentiment_label,Stock,source,url,relevance_score
Date,,,,,,
2023-03-27 10:00:00,0.181614,Somewhat-Bullish,RSSS,Benzinga,https://www.benzinga.com/pressreleases/23/03/n...,0.089129
2023-03-27 10:00:00,0.404327,Bullish,RAIL,Benzinga,https://www.benzinga.com/pressreleases/23/03/g...,0.500324
2023-03-27 10:00:00,0.19024,Somewhat-Bullish,GDVM,PennyStocks.com,https://pennystocks.com/featured/2023/03/27/3-...,0.026532
2023-03-27 07:26:12,0.446184,Bullish,FSR,Investing News Network,https://investingnews.com/all-electric-fisker-...,0.862645
2023-03-27 07:26:12,0.317112,Somewhat-Bullish,GOOG,Investing News Network,https://investingnews.com/all-electric-fisker-...,0.090989
...,...,...,...,...,...,...
2023-02-21 07:42:00,0.003875,Neutral,NWG,Reuters,https://www.reuters.com/markets/europe/hsbc-cu...,0.140076
2023-02-21 07:31:30,0.327663,Somewhat-Bullish,NIO,Benzinga,https://www.benzinga.com/news/23/02/30980568/n...,0.730954
2023-02-21 07:31:30,0.086742,Neutral,TSLA,Benzinga,https://www.benzinga.com/news/23/02/30980568/n...,0.472338


In [4]:
rds_key_pass = pd.read_csv('db_key_pass.csv')
username = rds_key_pass['ID'][0]
password = rds_key_pass['PASS'][0]
url = rds_key_pass['URL'][0]

In [5]:
# change column names to match RDS table
sentiment_ticker_list = sentiment_ticker_list.rename(columns={'Stock': 'stock', 'ticker_sentiment_score': 'sentiment',
                                                              'ticker_sentiment_label': 'sentiment_label', 'url': 'article_url'})

In [45]:
sentiment_ticker_list['time_published'] = sentiment_ticker_list.index
# drop duplicates
new_sents = sentiment_ticker_list.copy().drop_duplicates(subset=['stock', 'time_published']).set_index(['time_published','stock']).sort_index(ascending=False)
new_sents

sentiment   sentiment_label                  source  \
time_published      stock                                                       
2023-03-27 10:00:00 RSSS   0.181614  Somewhat-Bullish                Benzinga   
                    RAIL   0.404327           Bullish                Benzinga   
                    GDVM    0.19024  Somewhat-Bullish         PennyStocks.com   
2023-03-27 07:26:12 META   0.295401  Somewhat-Bullish  Investing News Network   
                    GOOG   0.317112  Somewhat-Bullish  Investing News Network   
...                             ...               ...                     ...   
2023-02-21 07:42:00 NWG    0.003875           Neutral                 Reuters   
2023-02-21 07:31:30 TSLA   0.086742           Neutral                Benzinga   
                    NIO    0.327663  Somewhat-Bullish                Benzinga   
2023-02-21 07:30:00 CVS    0.082247           Neutral           GlobeNewswire   
2023-02-21 07:05:00 DSSMY       0.0           Neutral                 Reuters   

                                                                 article_url  \
time_published      stock                                                      
2023-03-27 10:00:00 RSSS   https://www.benzinga.com/pressreleases/23/03/n...   
                    RAIL   https://www.benzinga.com/pressreleases/23/03/g...   
                    GDVM   https://pennystocks.com/featured/2023/03/27/3-...   
2023-03-27 07:26:12 META   https://investingnews.com/all-electric-fisker-...   
                    GOOG   https://investingnews.com/all-electric-fisker-...   
...                                                                      ...   
2023-02-21 07:42:00 NWG    https://www.reuters.com/markets/europe/hsbc-cu...   
2023-02-21 07:31:30 TSLA   https://www.benzinga.com/news/23/02/30980568/n...   
                    NIO    https://www.benzinga.com/news/23/02/30980568/n...   
2023-02-21 07:30:00 CVS    https://www.globenewswire.com/news-release/202...   
2023-02-21 07:05:00 DSSMY  https://www.reuters.com/markets/europe/resilie...   

                          relevance_score  
time_published      stock                  
2023-03-27 10:00:00 RSSS         0.089129  
                    RAIL         0.500324  
                    GDVM         0.026532  
2023-03-27 07:26:12 META         0.045569  
                    GOOG         0.090989  
...                                   ...  
2023-02-21 07:42:00 NWG          0.140076  
2023-02-21 07:31:30 TSLA         0.472338  
                    NIO          0.730954  
2023-02-21 07:30:00 CVS          0.010758  
2023-02-21 07:05:00 DSSMY        0.040416  

[12382 rows x 5 columns]

In [46]:
import pymysql
from sqlalchemy import create_engine
import sqlalchemy

In [84]:
dtypes = {'time_published': sqlalchemy.types.DATETIME, 'relevance_score': sqlalchemy.types.DECIMAL(6,5), 'sentiment': sqlalchemy.types.DECIMAL(6,5),
                                'stock': sqlalchemy.types.VARCHAR(10), 'sentiment_label': sqlalchemy.types.VARCHAR(10), 'article_url': sqlalchemy.types.VARCHAR(200), 'source': sqlalchemy.types.VARCHAR(50)}
# Establish connection to the MySQL database
with pymysql.connect(host=url, user=username, password=password, db='stock_data') as con:
    # Create a SQLAlchemy engine object
    engine = create_engine(f'mysql+pymysql://{username}:{password}@{url}/stock_data', echo=False)
    try:
        # Convert the pandas DataFrame to a MySQL table
        new_sents.iloc[:-5].to_sql(name='Sentiments', con=engine, if_exists='replace', index=True, index_label=['time_published', 'stock'], dtype=dtypes)
        con.cursor().execute('ALTER TABLE Sentiments ADD PRIMARY KEY (stock, time_published);')
    except Exception as e:
        print(e)

In [85]:
new_sents.iloc[-5:]

sentiment   sentiment_label         source  \
time_published      stock                                              
2023-02-21 07:42:00 NWG    0.003875           Neutral        Reuters   
2023-02-21 07:31:30 TSLA   0.086742           Neutral       Benzinga   
                    NIO    0.327663  Somewhat-Bullish       Benzinga   
2023-02-21 07:30:00 CVS    0.082247           Neutral  GlobeNewswire   
2023-02-21 07:05:00 DSSMY       0.0           Neutral        Reuters   

                                                                 article_url  \
time_published      stock                                                      
2023-02-21 07:42:00 NWG    https://www.reuters.com/markets/europe/hsbc-cu...   
2023-02-21 07:31:30 TSLA   https://www.benzinga.com/news/23/02/30980568/n...   
                    NIO    https://www.benzinga.com/news/23/02/30980568/n...   
2023-02-21 07:30:00 CVS    https://www.globenewswire.com/news-release/202...   
2023-02-21 07:05:00 DSSMY  https://www.reuters.com/markets/europe/resilie...   

                          relevance_score  
time_published      stock                  
2023-02-21 07:42:00 NWG          0.140076  
2023-02-21 07:31:30 TSLA         0.472338  
                    NIO          0.730954  
2023-02-21 07:30:00 CVS          0.010758  
2023-02-21 07:05:00 DSSMY        0.040416

In [86]:
dtypes = {'time_published': sqlalchemy.types.DATETIME, 'relevance_score': sqlalchemy.types.DECIMAL(6,5), 'sentiment': sqlalchemy.types.DECIMAL(6,5),
                                'stock': sqlalchemy.types.VARCHAR(10), 'sentiment_label': sqlalchemy.types.VARCHAR(10), 'article_url': sqlalchemy.types.VARCHAR(200), 'source': sqlalchemy.types.VARCHAR(50)}
# Establish connection to the MySQL database
with pymysql.connect(host=url, user=username, password=password, db='stock_data') as con:
    # Create a SQLAlchemy engine object
    for i in range(len(new_sents.iloc[-10:])):
        # print(new_sents.loc[idx])
        engine = create_engine(f'mysql+pymysql://{username}:{password}@{url}/stock_data', echo=False)
        try:
            # Convert the pandas DataFrame to a MySQL table
            new_sents.iloc[-10:].iloc[i:i+1].to_sql(name='Sentiments', con=engine, if_exists='append', index=True, index_label=['time_published', 'stock'], dtype=dtypes)
        except Exception as e:
            print(e)

(pymysql.err.IntegrityError) (1062, "Duplicate entry 'FOREX:USD-2023-02-21 07:50:00' for key 'Sentiments.PRIMARY'")
[SQL: INSERT INTO `Sentiments` (time_published, stock, sentiment, sentiment_label, source, article_url, relevance_score) VALUES (%(time_published)s, %(stock)s, %(sentiment)s, %(sentiment_label)s, %(source)s, %(article_url)s, %(relevance_score)s)]
[parameters: {'time_published': datetime.datetime(2023, 2, 21, 7, 50), 'stock': 'FOREX:USD', 'sentiment': '-0.165678', 'sentiment_label': 'Somewhat-Bearish', 'source': 'Reuters', 'article_url': 'https://www.reuters.com/markets/currencies/russian-rouble-weakens-ahead-putin-address-parliament-2023-02-21/', 'relevance_score': '0.233556'}]
(Background on this error at: https://sqlalche.me/e/20/gkpj)
(pymysql.err.IntegrityError) (1062, "Duplicate entry 'FOREX:EUR-2023-02-21 07:50:00' for key 'Sentiments.PRIMARY'")
[SQL: INSERT INTO `Sentiments` (time_published, stock, sentiment, sentiment_label, source, article_url, relevance_score) V

In [78]:
temp = new_sents.iloc[-10:]
temp

sentiment   sentiment_label  \
time_published      stock                                     
2023-02-21 07:50:00 FOREX:USD   -0.165678  Somewhat-Bearish   
                    FOREX:EUR   -0.127204           Neutral   
                    FOREX:CNY   -0.127204           Neutral   
2023-02-21 07:49:00 HAITY             0.0           Neutral   
2023-02-21 07:46:01 CRYPTO:BTC   0.013812           Neutral   
2023-02-21 07:42:00 NWG          0.003875           Neutral   
2023-02-21 07:31:30 TSLA         0.086742           Neutral   
                    NIO          0.327663  Somewhat-Bullish   
2023-02-21 07:30:00 CVS          0.082247           Neutral   
2023-02-21 07:05:00 DSSMY             0.0           Neutral   

                                               source  \
time_published      stock                               
2023-02-21 07:50:00 FOREX:USD                 Reuters   
                    FOREX:EUR                 Reuters   
                    FOREX:CNY                 Reuters   
2023-02-21 07:49:00 HAITY           Business Standard   
2023-02-21 07:46:01 CRYPTO:BTC  The Financial Express   
2023-02-21 07:42:00 NWG                       Reuters   
2023-02-21 07:31:30 TSLA                     Benzinga   
                    NIO                      Benzinga   
2023-02-21 07:30:00 CVS                 GlobeNewswire   
2023-02-21 07:05:00 DSSMY                     Reuters   

                                                                      article_url  \
time_published      stock                                                           
2023-02-21 07:50:00 FOREX:USD   https://www.reuters.com/markets/currencies/rus...   
                    FOREX:EUR   https://www.reuters.com/markets/currencies/rus...   
                    FOREX:CNY   https://www.reuters.com/markets/currencies/rus...   
2023-02-21 07:49:00 HAITY       https://www.business-standard.com/article/mark...   
2023-02-21 07:46:01 CRYPTO:BTC  https://www.financialexpress.com/blockchain/cr...   
2023-02-21 07:42:00 NWG         https://www.reuters.com/markets/europe/hsbc-cu...   
2023-02-21 07:31:30 TSLA        https://www.benzinga.com/news/23/02/30980568/n...   
                    NIO         https://www.benzinga.com/news/23/02/30980568/n...   
2023-02-21 07:30:00 CVS         https://www.globenewswire.com/news-release/202...   
2023-02-21 07:05:00 DSSMY       https://www.reuters.com/markets/europe/resilie...   

                               relevance_score  
time_published      stock                       
2023-02-21 07:50:00 FOREX:USD         0.233556  
                    FOREX:EUR         0.118064  
                    FOREX:CNY         0.118064  
2023-02-21 07:49:00 HAITY             0.200688  
2023-02-21 07:46:01 CRYPTO:BTC        0.406745  
2023-02-21 07:42:00 NWG               0.140076  
2023-02-21 07:31:30 TSLA              0.472338  
                    NIO               0.730954  
2023-02-21 07:30:00 CVS               0.010758  
2023-02-21 07:05:00 DSSMY             0.040416

In [79]:
for i in range(len(temp)):
    print(temp.iloc[i:i+1])

                               sentiment   sentiment_label   source  \
time_published      stock                                             
2023-02-21 07:50:00 FOREX:USD  -0.165678  Somewhat-Bearish  Reuters   

                                                                     article_url  \
time_published      stock                                                          
2023-02-21 07:50:00 FOREX:USD  https://www.reuters.com/markets/currencies/rus...   

                              relevance_score  
time_published      stock                      
2023-02-21 07:50:00 FOREX:USD        0.233556  
                               sentiment sentiment_label   source  \
time_published      stock                                           
2023-02-21 07:50:00 FOREX:EUR  -0.127204         Neutral  Reuters   

                                                                     article_url  \
time_published      stock                                                          
2023-02-21 07

In [4]:
# Connect to the database
connection = pymysql.connect(
    host=url,
    user=username,
    passwd=password,
    db="stock_data"
)
# load from RDS into pandas
sentiment_ticker_list = pd.read_sql('SELECT * FROM Sentiments', con=connection)

C:\Users\David\AppData\Local\Temp\ipykernel_27760\999920332.py:9: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  sentiment_ticker_list = pd.read_sql('SELECT * FROM Sentiments', con=connection)


DatabaseError: Execution failed on sql 'SELECT * FROM Sentiments': (1146, "Table 'stock_data.Sentiments' doesn't exist")

In [8]:
new_sents = sentiment_ticker_list.copy()
new_sents

,stock,relevance_score,sentiment,sentiment_label,article_url,source,time_published
0,AIMC,0.02,0.00,Neutral,https://www.prnewswire.com/news-releases/regal...,PR Newswire,2023-03-27 12:27:00
1,CURN,0.02,0.05,Neutral,https://www.prnewswire.com/news-releases/regal...,PR Newswire,2023-03-27 12:27:00
2,RRX,0.41,0.32,Somewhat-Bullish,https://www.prnewswire.com/news-releases/regal...,PR Newswire,2023-03-27 12:27:00
3,GE,0.01,0.13,Neutral,https://www.prnewswire.com/news-releases/regal...,PR Newswire,2023-03-27 12:27:00
4,ROK,0.01,-0.01,Neutral,https://www.prnewswire.com/news-releases/regal...,PR Newswire,2023-03-27 12:27:00
...,...,...,...,...,...,...,...
14453,GS,0.12,-0.09,Neutral,https://www.axios.com/2023/03/29/office-real-e...,Axios,2023-03-29 11:20:00
14454,ENOV,0.38,0.32,Somewhat-Bullish,https://www.benzinga.com/trading-ideas/long-id...,Benzinga,2023-03-29 11:28:50
14455,ESPR,0.16,0.19,Somewhat-Bullish,https://www.benzinga.com/trading-ideas/long-id...,Benzinga,2023-03-29 11:28:50
14456,SLS,0.24,0.05,Neutral,https://www.benzinga.com/trading-ideas/long-id...,Benzinga,2023-03-29 11:28:50


In [10]:
# drop duplicates
new_sents = new_sents.drop_duplicates(subset=['stock', 'time_published']).set_index(['stock', 'time_published']).sort_index()
new_sents

relevance_score  sentiment   sentiment_label  \
stock time_published                                                      
A     2023-03-08 14:00:00             0.13       0.14           Neutral   
      2023-03-14 19:30:00             0.05       0.00           Neutral   
AA    2023-03-14 22:00:20             0.67       0.26  Somewhat-Bullish   
AABB  2023-03-09 13:00:00             0.75       0.35           Bullish   
AADI  2023-03-10 00:25:04             0.21       0.00           Neutral   
...                                    ...        ...               ...   
ZVRA  2023-03-15 11:26:12             0.08       0.08           Neutral   
ZYNE  2023-03-01 12:00:00             0.21      -0.01           Neutral   
ZYXI  2023-03-13 14:55:50             0.16       0.53           Bullish   
      2023-03-15 22:52:00             0.44       0.03           Neutral   
      2023-03-20 23:08:33             0.62       0.00           Neutral   

                                                                 article_url  \
stock time_published                                                           
A     2023-03-08 14:00:00  https://www.benzinga.com/pressreleases/23/03/g...   
      2023-03-14 19:30:00  https://www.benzinga.com/pressreleases/23/03/g...   
AA    2023-03-14 22:00:20  https://www.zacks.com/stock/news/2065773/alcoa...   
AABB  2023-03-09 13:00:00  https://www.benzinga.com/pressreleases/23/03/g...   
AADI  2023-03-10 00:25:04  https://www.zacks.com/stock/news/2063886/carib...   
...                                                                      ...   
ZVRA  2023-03-15 11:26:12  https://www.benzinga.com/pressreleases/23/03/g...   
ZYNE  2023-03-01 12:00:00  https://www.benzinga.com/pressreleases/23/03/g...   
ZYXI  2023-03-13 14:55:50  https://www.benzinga.com/news/23/03/31322709/i...   
      2023-03-15 22:52:00  https://www.benzinga.com/pressreleases/23/03/g...   
      2023-03-20 23:08:33  https://www.benzinga.com/pressreleases/23/03/g...   

                                     source  
stock time_published                         
A     2023-03-08 14:00:00          Benzinga  
      2023-03-14 19:30:00          Benzinga  
AA    2023-03-14 22:00:20  Zacks Commentary  
AABB  2023-03-09 13:00:00          Benzinga  
AADI  2023-03-10 00:25:04  Zacks Commentary  
...                                     ...  
ZVRA  2023-03-15 11:26:12          Benzinga  
ZYNE  2023-03-01 12:00:00          Benzinga  
ZYXI  2023-03-13 14:55:50          Benzinga  
      2023-03-15 22:52:00          Benzinga  
      2023-03-20 23:08:33          Benzinga  

[12953 rows x 5 columns]

In [12]:
# Establish connection to the MySQL database
conn = pymysql.connect(host=url, user=username, password=password, db='stock_data')

# Create a SQLAlchemy engine object
engine = create_engine(f'mysql+pymysql://{username}:{password}@{url}/stock_data', echo=False)

# Convert the pandas DataFrame to a MySQL table
new_sents.to_sql(name='Sentiments', con=engine, if_exists='replace', index=False, dtype={'time_published': sqlalchemy.types.DATETIME, 'relevance_score': sqlalchemy.types.DECIMAL(6,5), 'sentiment': sqlalchemy.types.DECIMAL(6,5)})

# Close the connection
conn.close()